#
# PART 1: INSTALLATION AND SETUP
#

In [2]:
!pip install "supabase>=2.5" sentence-transformers torch --extra-index-url https://download.pytorch.org/whl/cpu
from supabase import create_client
from sentence_transformers import SentenceTransformer
import requests
import json
import pandas as pd
from datetime import datetime

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00


# Supabase Configuration


In [ ]:
SUPABASE_URL =
SUPABASE_SERVICE_KEY = 

# Initialize Supabase client and embedding model

In [4]:
client = create_client(SUPABASE_URL, SUPABASE_SERVICE_KEY)
model = SentenceTransformer('all-MiniLM-L6-v2')

print("✅ Connected to Supabase and model loaded")
print(f"📊 Model dimension: {model.get_sentence_embedding_dimension()}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Connected to Supabase and model loaded
📊 Model dimension: 384


# =============================================
# PART 2: FAQ DATASET - iTech Solutions
# =============================================

# =============================================
# PART 2: FAQ DATASET - iTech Solutions
# =============================================

In [5]:

faq_data = [
    {
        "question": "Who is iTech Solutions?",
        "answer": "iTech Solutions is a Google Premier Partner and one of the oldest technology solution providers in the Arab region."
    },
    {
        "question": "Where does iTech operate?",
        "answer": "We serve clients across the Gulf, the United States, and Europe."
    },
    {
        "question": "What services does iTech provide?",
        "answer": "We deliver innovative technology and AI-driven solutions, including cloud migration, DevOps, web and mobile development, and IT consulting."
    },
    {
        "question": "What industries does iTech support?",
        "answer": "We work with enterprises, startups, and institutions across multiple industries to drive digital transformation."
    },
    {
        "question": "What makes iTech unique?",
        "answer": "We combine decades of experience with advanced AI and cloud solutions, and we are a Google Premier Partner."
    },
    {
        "question": "Does iTech offer training or onboarding?",
        "answer": "Yes, we run a two-month fast-track onboarding program to integrate engineers directly into our AI Engineering team."
    },
    {
        "question": "Who will I work with at iTech?",
        "answer": "You will collaborate with a multinational team, including professionals from Microsoft and other global tech leaders, as well as colleagues from top universities."
    },
    {
        "question": "How can I contact iTech support?",
        "answer": "You can email support@itech.example or call +20-123-456-789."
    },
    {
        "question": "What is iTech's typical project timeline?",
        "answer": "An MVP can usually be delivered in 4–8 weeks depending on scope."
    },
    {
        "question": "Does iTech offer long-term maintenance?",
        "answer": "Yes, we provide monthly and annual support and maintenance packages with SLAs."
    }
]

print(f"📋 Loaded {len(faq_data)} FAQ entries")

📋 Loaded 10 FAQ entries


# =============================================
# PART 3: EMBEDDING-BASED FAQ BOT (PYTHON)
# =============================================

In [6]:

def setup_embedding_faq_bot():
    """Setup embedding-based FAQ bot with vector search"""

    # Create embeddings for all questions
    questions = [row["question"] for row in faq_data]
    embeddings = model.encode(questions, normalize_embeddings=True)

    # Prepare data for insertion
    rows = []
    for row, embedding in zip(faq_data, embeddings):
        rows.append({
            "question": row["question"],
            "answer": row["answer"],
            "embedding": embedding.tolist()
        })

    try:
        # Insert data into Supabase
        response = client.table("faq").insert(rows).execute()
        print(f"✅ Successfully inserted {len(response.data)} FAQ entries with embeddings")
        return True
    except Exception as e:
        print(f"❌ Error inserting data: {e}")
        return False

def ask_faq_embeddings(query, k=3):
    """Query FAQ using embedding-based vector search"""
    try:
        # Generate embedding for user query
        query_embedding = model.encode([query], normalize_embeddings=True)[0].tolist()

        # Call Supabase RPC function for vector search
        result = client.rpc("match_faqs", {
            "query_embedding": query_embedding,
            "match_count": k
        }).execute()

        return result.data
    except Exception as e:
        print(f"❌ Error in embedding search: {e}")
        return []

# Setup the embedding-based bot
print("\n" + "="*60)
print("SETTING UP EMBEDDING-BASED FAQ BOT")
print("="*60)

embedding_setup_success = setup_embedding_faq_bot()


SETTING UP EMBEDDING-BASED FAQ BOT
✅ Successfully inserted 10 FAQ entries with embeddings


# =============================================
# PART 4: KEYWORD-BASED FAQ BOT (N8N COMPATIBLE)
# =============================================

In [7]:
def ask_faq_keywords(query, match_count=3):
    """Query FAQ using keyword-based search (compatible with n8n)"""

    url = f"{SUPABASE_URL}/rest/v1/rpc/search_faqs_simple"
    headers = {
        "apikey": SUPABASE_SERVICE_KEY,
        "Authorization": f"Bearer {SUPABASE_SERVICE_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "search_query": query,
        "match_count": match_count
    }

    try:
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"❌ Error in keyword search: {e}")
        return []

# =============================================
# PART 5: COMPREHENSIVE TESTING
# =============================================


In [8]:
test_questions = [
    "What services does iTech offer?",
    "How can I contact support?",
    "Do you have training for new engineers?",
    "What makes iTech unique?",
    "Where do you operate?",
    "What is your project timeline?",
    "Do you offer maintenance?",
    "Who will I work with?"
]

print("\n" + "="*60)
print("COMPREHENSIVE FAQ BOT TESTING")
print("="*60)

results_comparison = []

for question in test_questions:
    print(f"\n🤔 Question: {question}")
    print("-" * 50)

    # Test embedding-based search
    if embedding_setup_success:
        embedding_results = ask_faq_embeddings(question, k=2)
        print("📊 EMBEDDING-BASED RESULTS:")
        if embedding_results:
            for i, result in enumerate(embedding_results, 1):
                similarity_percentage = round(result['similarity'] * 100, 1)
                print(f"   {i}. [{similarity_percentage}%] {result['answer'][:100]}...")
        else:
            print("   ❌ No embedding results found")

    # Test keyword-based search
    keyword_results = ask_faq_keywords(question, match_count=2)
    print("\n🔍 KEYWORD-BASED RESULTS (N8N Compatible):")
    if keyword_results:
        for i, result in enumerate(keyword_results, 1):
            score = result.get('match_score', 0)
            print(f"   {i}. [Score: {score}] {result['answer'][:100]}...")
    else:
        print("   ❌ No keyword results found")

    # Store results for comparison
    results_comparison.append({
        "question": question,
        "embedding_count": len(embedding_results) if embedding_setup_success and embedding_results else 0,
        "keyword_count": len(keyword_results) if keyword_results else 0,
        "timestamp": datetime.now().isoformat()
    })


COMPREHENSIVE FAQ BOT TESTING

🤔 Question: What services does iTech offer?
--------------------------------------------------
📊 EMBEDDING-BASED RESULTS:
   1. [96.7%] We deliver innovative technology and AI-driven solutions, including cloud migration, DevOps, web and...
   2. [96.7%] We deliver innovative technology and AI-driven solutions, including cloud migration, DevOps, web and...

🔍 KEYWORD-BASED RESULTS (N8N Compatible):
   ❌ No keyword results found

🤔 Question: How can I contact support?
--------------------------------------------------
📊 EMBEDDING-BASED RESULTS:
   1. [65.9%] You can email support@itech.example or call +20-123-456-789....
   2. [65.9%] You can email support@itech.example or call +20-123-456-789....

🔍 KEYWORD-BASED RESULTS (N8N Compatible):
   ❌ No keyword results found

🤔 Question: Do you have training for new engineers?
--------------------------------------------------
📊 EMBEDDING-BASED RESULTS:
   1. [49.4%] Yes, we run a two-month fast-track onboarding

# =============================================
# PART 6: PERFORMANCE ANALYSIS
# =============================================


In [9]:

print("\n" + "="*60)
print("PERFORMANCE ANALYSIS")
print("="*60)

# Create performance DataFrame
df_results = pd.DataFrame(results_comparison)
print("\n📊 Results Summary:")
print(df_results[['question', 'embedding_count', 'keyword_count']])

print(f"\n📈 Statistics:")
print(f"   • Average embedding results: {df_results['embedding_count'].mean():.2f}")
print(f"   • Average keyword results: {df_results['keyword_count'].mean():.2f}")
print(f"   • Questions with embedding results: {(df_results['embedding_count'] > 0).sum()}/{len(df_results)}")
print(f"   • Questions with keyword results: {(df_results['keyword_count'] > 0).sum()}/{len(df_results)}")


PERFORMANCE ANALYSIS

📊 Results Summary:
                                  question  embedding_count  keyword_count
0          What services does iTech offer?                2              0
1               How can I contact support?                2              0
2  Do you have training for new engineers?                2              0
3                 What makes iTech unique?                2              2
4                    Where do you operate?                2              0
5           What is your project timeline?                2              0
6                Do you offer maintenance?                2              0
7                    Who will I work with?                2              0

📈 Statistics:
   • Average embedding results: 2.00
   • Average keyword results: 0.25
   • Questions with embedding results: 8/8
   • Questions with keyword results: 1/8


# =============================================
# PART 7: N8N INTEGRATION DEMO
# =============================================


In [10]:
print("\n" + "="*60)
print("N8N INTEGRATION DEMONSTRATION")
print("="*60)

def simulate_n8n_webhook_call(question):
    """Simulate what happens when n8n calls the FAQ bot"""
    print(f"\n🔗 Simulating n8n webhook call...")
    print(f"   Incoming request: {{'question': '{question}'}}")

    # This is what the n8n HTTP Request node would do
    results = ask_faq_keywords(question, match_count=1)

    if results and len(results) > 0:
        response = {
            "question": question,
            "answer": results[0]["answer"],
            "confidence": results[0]["match_score"],
            "timestamp": datetime.now().isoformat()
        }
    else:
        response = {
            "question": question,
            "answer": "I'm sorry, I couldn't find an answer to your question. Please contact support at support@itech.example or call +20-123-456-789.",
            "confidence": 0,
            "timestamp": datetime.now().isoformat()
        }

    print(f"   Response: {json.dumps(response, indent=2)}")
    return response

# Test n8n simulation
n8n_test_questions = [
    "What services does iTech provide?",
    "How can I contact support?",
    "Random question that won't match"
]

for test_q in n8n_test_questions:
    simulate_n8n_webhook_call(test_q)


N8N INTEGRATION DEMONSTRATION

🔗 Simulating n8n webhook call...
   Incoming request: {'question': 'What services does iTech provide?'}
   Response: {
  "question": "What services does iTech provide?",
  "answer": "We deliver innovative technology and AI-driven solutions, including cloud migration, DevOps, web and mobile development, and IT consulting.",
  "confidence": 10,
  "timestamp": "2025-09-03T20:12:54.608374"
}

🔗 Simulating n8n webhook call...
   Incoming request: {'question': 'How can I contact support?'}
   Response: {
  "question": "How can I contact support?",
  "answer": "I'm sorry, I couldn't find an answer to your question. Please contact support at support@itech.example or call +20-123-456-789.",
  "confidence": 0,
  "timestamp": "2025-09-03T20:12:54.888894"
}

🔗 Simulating n8n webhook call...
   Incoming request: {'question': 'Random question that won't match'}
   Response: {
  "question": "Random question that won't match",
  "answer": "I'm sorry, I couldn't find an 

# =============================================
# PART 8: DIRECT API TESTING (FOR DEBUGGING)
# =============================================

In [11]:
print("DIRECT API TESTING")
print("="*60)

def test_supabase_direct():
    """Test direct Supabase API calls"""

    # Test embedding search
    print("🧪 Testing embedding-based search...")
    try:
        query = "What services does iTech provide?"
        query_embedding = model.encode([query], normalize_embeddings=True)[0].tolist()

        url = f"{SUPABASE_URL}/rest/v1/rpc/match_faqs"
        headers = {
            "apikey": SUPABASE_SERVICE_KEY,
            "Authorization": f"Bearer {SUPABASE_SERVICE_KEY}",
            "Content-Type": "application/json"
        }

        data = {
            "query_embedding": query_embedding,
            "match_count": 1
        }

        response = requests.post(url, headers=headers, data=json.dumps(data))
        print(f"   Status: {response.status_code}")
        print(f"   Result: {response.json()}")

    except Exception as e:
        print(f"   ❌ Embedding search error: {e}")

    # Test keyword search
    print("\n🔍 Testing keyword-based search...")
    try:
        url = f"{SUPABASE_URL}/rest/v1/rpc/search_faqs_simple"
        headers = {
            "apikey": SUPABASE_SERVICE_KEY,
            "Authorization": f"Bearer {SUPABASE_SERVICE_KEY}",
            "Content-Type": "application/json"
        }

        data = {
            "search_query": "services",
            "match_count": 1
        }

        response = requests.post(url, headers=headers, data=json.dumps(data))
        print(f"   Status: {response.status_code}")
        print(f"   Result: {response.json()}")

    except Exception as e:
        print(f"   ❌ Keyword search error: {e}")

test_supabase_direct()

DIRECT API TESTING
🧪 Testing embedding-based search...
   Status: 200
   Result: [{'id': 3, 'question': 'What services does iTech provide?', 'answer': 'We deliver innovative technology and AI-driven solutions, including cloud migration, DevOps, web and mobile development, and IT consulting.', 'similarity': 1}]

🔍 Testing keyword-based search...
   Status: 200
   Result: [{'id': 3, 'question': 'What services does iTech provide?', 'answer': 'We deliver innovative technology and AI-driven solutions, including cloud migration, DevOps, web and mobile development, and IT consulting.', 'match_score': 10}]
